In [1]:
import datetime
import time as time_module
import sys
import os 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import theano
import matplotlib
import pymc3 as pm
import theano.tensor as tt

try: 
    import covid19_inference as cov19
except ModuleNotFoundError:
    sys.path.append('../../../')
    import covid19_inference as cov19

## Global Parameters for all countries
Get all parameters which is needed for each of the non hierachical model runs. And create a jhu data instance

In [2]:
%run ../countries_parameters.ipynb #Retrieves the country dependent cp and population numbers 

In [3]:
jhu = cov19.data_retrieval.JHU(True)

INFO     [covid19_inference.data_retrieval._JHU] Successfully loaded data from local


In [4]:
data_begin = datetime.datetime(2020, 1, 23)
diff_data_sim = 16
num_days_forecast = 10

### Construct new_cases

In [5]:
new_cases = pd.DataFrame()
for key in countries:
    new_cases[key] = jhu.get_new(value="confirmed",country=key,data_begin=data_begin)
new_cases_obs = np.array(new_cases)

### Construct population array 

In [6]:
N_population = [countries[key]["N_population"] for key in countries]

### Construct changepoints

In [7]:
change_points = [countries[key]["change_points"] for key in countries]

We create the parameters for the model.

In [8]:
params_model = dict(new_cases_obs = new_cases_obs,
                    data_begin = data_begin,
                    fcast_len = num_days_forecast,
                    diff_data_sim = diff_data_sim,
                    N_population = N_population)

with cov19.Cov19Model(**params_model) as model:
    lambda_t_log = cov19.lambda_t_with_sigmoids(
                                pr_median_lambda_0 = 0.4,
                                change_points_list = change_points)
    
    # set prior distribution for the recovery rate
    mu = pm.Lognormal(name="mu", mu=np.log(1/8), sigma=0.2)
    pr_median_delay = 10
    
    # This builds a decorrelated prior for I_begin for faster inference. 
    # It is not necessary to use it, one can simply remove it and use the default argument 
    # for pr_I_begin in cov19.SIR
    prior_I = cov19.make_prior_I(lambda_t_log, mu, pr_median_delay = pr_median_delay)
    
    # Use lambda_t_log and mu to run the SIR model
    new_I_t = cov19.SIR(lambda_t_log, mu, pr_I_begin = prior_I)

    # Delay the cases by a lognormal reporting delay
    new_cases_inferred_raw = cov19.delay_cases(new_I_t, pr_median_delay=pr_median_delay, 
                                               pr_median_scale_delay=0.3)
    
    # Modulate the inferred cases by a abs(sin(x)) function, to account for weekend effects
    new_cases_inferred = cov19.week_modulation(new_cases_inferred_raw)
    
    # Define the likelihood, uses the new_cases_obs set as model parameter
    cov19.student_t_likelihood(new_cases_inferred)

AttributeError: 'list' object has no attribute 'keys'

MCMC sampling

In [24]:
trace = pm.sample(model=model, tune=500, draws=1200, init='advi+adapt_diag')

Auto-assigning NUTS sampler...
INFO     [pymc3] Auto-assigning NUTS sampler...
Initializing NUTS using advi+adapt_diag...
INFO     [pymc3] Initializing NUTS using advi+adapt_diag...
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
WARNING  [theano.tensor.blas] We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
WARNING  [theano.tensor.blas] We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
Average Loss = 1,190.2:  66%|██████▋   | 132614/200000 [07:10<03:38, 308.34it/s]
Interrupted at 13

ValueError: Not enough samples to build a trace.